# Extracting Insights from Mental Wellness App Reviews - Data Collection

* Name: Jonathan Lee
* Email: lee.hanjin@gmail.com

## TABLE OF CONTENTS 


- **[Introduction](#INTRODUCTION)<br>**
- **[OBTAIN](#OBTAIN)**<br>
- **[SCRUB](#SCRUB)**<br>
- **[EXPLORE](#EXPLORE)**<br>
- **[MODEL](#MODEL)**<br>
- **[iNTERPRET](#iNTERPRET)**<br>
- **[Conclusions/Recommendations](#CONCLUSIONS-&-RECOMMENDATIONS)<br>**
___

## INTRODUCTION

> In this notebook, we take a look at how to use the [itunes_app_scraper](https://pypi.org/project/itunes-app-scraper-dmi/) and [app_store_scraper](https://pypi.org/project/app-store-scraper/) library to collect data on apps and app reviews for a hand-curated list of mental wellness apps on the iTunes app store. An excellent guide to using these packages is provided by Max Steele [here](https://python.plainenglish.io/scraping-app-store-reviews-with-python-90e4117ccdfb).

## Importing Packages

In [1]:
# Standard packages for manipulating dataframes and keeping track of 
# scraping times.
import pandas as pd
import datetime as dt
from tzlocal import get_localzone
import random
import time

# Package for scraping app info
from itunes_app_scraper.scraper import AppStoreScraper

# Package for scraping app reviews
from app_store_scraper import AppStore

## Loading in App Names and App IDs
Since there is no category for "Mental Wellness" or "Mental Health" in the iTunes App Store, I have manually created a list of apps with their corresponding app IDs by doing a series of Google searches for the best mental wellness/mental health apps available. We can see this information along with approximately how many reviews each app has on the iTunes App Store.

In [2]:
# Set no limit to the number of max columns to display in pandas
pd.set_option('display.max_columns', None)

In [3]:
# Load file containing app names and IDs
app_df = pd.read_csv('data/app-name-id.csv')
app_df

,app_name,ios_app_name,ios_app_id,ios_reviews,google_app_id,google_reviews
0,Headspace,headspace-meditation-sleep,493145008,"806,200",com.getsomeheadspace.android,"231,603"
1,Calm,calm,571800810,"1,300,000",com.calm.android,"408,135"
2,Stoic,stoic,1312926037,87,com.stoicroutine.stoic,377
3,Happify,happify-for-stress-worry,730601963,"3,400",com.happify.happifyinc,"2,764"
4,Shine,shine-calm-anxiety-stress,1293721589,"28,800",com.shinetext.shine,"8,221"
5,MindShift,mindshift-cbt-anxiety-relief,634684825,211,com.bstro.MindShift,"1,438"
6,MoodTools,moodtools-depression-aid,1012822112,127,com.moodtools.moodtools,"3,169"
7,Insight Timer,insight-timer-meditation-app,337472899,"307,100",com.spotlightsix.zentimerlite2,"129,194"
8,Moodfit,moodfit,1054458809,329,com.robleridge.Moodfit,504
9,Talkspace,talkspace-therapy-counseling,661829386,"20,400",com.talkspace.talkspaceapp,"3,551"


## Scraping App Info
Here, we use the itunes_app_scraper package to scrape data regarding each of our apps. Although our analysis will focus on Natural Language Processing of the reviews, this data could come in handy during our analysis, so we will go ahead and save this data as its own .csv file.

In [4]:
# Create lists of app names and app IDs
app_names = list(app_df['ios_app_name'])
app_ids = list(app_df['ios_app_id'])

In [5]:
# Preview app_names list
app_names

['headspace-meditation-sleep',
 'calm',
 'stoic',
 'happify-for-stress-worry',
 'shine-calm-anxiety-stress',
 'mindshift-cbt-anxiety-relief',
 'moodtools-depression-aid',
 'insight-timer-meditation-app',
 'moodfit',
 'talkspace-therapy-counseling',
 'moodmission',
 'sanvello-anxiety-depression',
 'minddoc-your-companion',
 'smiling-mind',
 'cbt-i-coach',
 'youper-self-guided-therapy',
 'innerhour-self-care-therapy',
 'breathe2relax',
 'rootd-panic-attack-relief',
 'ten-percent-happier-meditation',
 'pzizz-sleep-nap-focus',
 'noisli',
 'slumber-fall-asleep-insomnia',
 'whats-up-a-mental-health-app',
 'breethe-meditation-sleep',
 'meditopia-meditation-breathe',
 'betterme-calm-sleep-meditate',
 'mindfulness-with-petit-bambou',
 'fabulous-daily-routine-planner',
 'reflectly-journal-ai-diary',
 'relax-meditation-guided-mind']

In [6]:
# Initialize App Store Scraper and scrape data
scraper = AppStoreScraper()
app_store_list = list(scraper.get_multiple_app_details(app_ids))

https://itunes.apple.com/lookup?id=493145008&country=nl&entity=software
https://itunes.apple.com/lookup?id=571800810&country=nl&entity=software
https://itunes.apple.com/lookup?id=1312926037&country=nl&entity=software
https://itunes.apple.com/lookup?id=730601963&country=nl&entity=software
https://itunes.apple.com/lookup?id=1293721589&country=nl&entity=software
https://itunes.apple.com/lookup?id=634684825&country=nl&entity=software
https://itunes.apple.com/lookup?id=1012822112&country=nl&entity=software
https://itunes.apple.com/lookup?id=337472899&country=nl&entity=software
https://itunes.apple.com/lookup?id=1054458809&country=nl&entity=software
https://itunes.apple.com/lookup?id=661829386&country=nl&entity=software
https://itunes.apple.com/lookup?id=1140332763&country=nl&entity=software
https://itunes.apple.com/lookup?id=922968861&country=nl&entity=software
https://itunes.apple.com/lookup?id=1052216403&country=nl&entity=software
https://itunes.apple.com/lookup?id=560442518&country=nl&en

In [7]:
# Convert list of dicts to Pandas DataFrame and write to csv
app_info_df = pd.DataFrame(app_store_list)
app_info_df.to_csv('data/app_info.csv', index=False)
app_info_df

,screenshotUrls,ipadScreenshotUrls,appletvScreenshotUrls,artworkUrl60,artworkUrl512,artworkUrl100,artistViewUrl,features,supportedDevices,advisories,isGameCenterEnabled,kind,minimumOsVersion,trackCensoredName,languageCodesISO2A,fileSizeBytes,sellerUrl,formattedPrice,contentAdvisoryRating,averageUserRatingForCurrentVersion,userRatingCountForCurrentVersion,averageUserRating,trackViewUrl,trackContentRating,bundleId,genreIds,primaryGenreName,trackId,trackName,sellerName,isVppDeviceBasedLicensingEnabled,currentVersionReleaseDate,releaseNotes,primaryGenreId,currency,description,artistId,artistName,genres,price,releaseDate,version,wrapperType,userRatingCount
0,https://is2-ssl.mzstatic.com/image/thumb/Purpl...,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,https://apps.apple.com/nl/developer/headspace-...,iosUniversal,"iPhone5s-iPhone5s,iPadAir-iPadAir,iPadAirCellu...",,False,software,13.0,Headspace: Meditation & Sleep,"EN,FR,DE,PT,ES",104807424,https://www.headspace.com/,Gratis,4+,4.73281,41094,4.73281,https://apps.apple.com/nl/app/headspace-medita...,4+,com.getsomeheadspace.headspace,"6013,6007",Health & Fitness,493145008,Headspace: Meditation & Sleep,Headspace Inc.,True,2021-06-29T22:31:11Z,Thanks for using Headspace! This update includ...,6013,EUR,Get happy. Stress less. Sleep soundly. Headspa...,384434796,Headspace Inc.,"Gezondheid en fitness,Productiviteit",0.00,2012-02-02T18:57:49Z,3.170.0,software,41094
1,https://is2-ssl.mzstatic.com/image/thumb/Purpl...,https://is4-ssl.mzstatic.com/image/thumb/Purpl...,,https://is4-ssl.mzstatic.com/image/thumb/Purpl...,https://is4-ssl.mzstatic.com/image/thumb/Purpl...,https://is4-ssl.mzstatic.com/image/thumb/Purpl...,https://apps.apple.com/nl/developer/calm-com/i...,iosUniversal,"iPhone5s-iPhone5s,iPadAir-iPadAir,iPadAirCellu...",,False,software,13.0,Calm,"EN,FR,DE,JA,KO,PT,ES",111981568,https://www.calm.com,Gratis,4+,4.59842,13935,4.59842,https://apps.apple.com/nl/app/calm/id571800810...,4+,com.calm.calmapp,"6013,6012",Health & Fitness,571800810,Calm,"Calm.com, Inc.",True,2021-07-01T14:00:15Z,Thanks for using Calm! This update contains mu...,6013,EUR,** Best of 2018 Award Winner ** — Apple\n\n** ...,571800813,Calm.com,"Gezondheid en fitness,Lifestyle",0.00,2013-02-22T20:09:23Z,5.21,software,13935
2,https://is2-ssl.mzstatic.com/image/thumb/Purpl...,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,https://apps.apple.com/nl/developer/maciej-lob...,iosUniversal,"MacDesktop-MacDesktop,iPhone5s-iPhone5s,iPadAi...",,False,software,13.0,stoic.,"EN,FR,DE,IT,PL,ES,SV",72890368,http://stoicroutine.com/,Gratis,4+,4.73475,328,4.73475,https://apps.apple.com/nl/app/stoic/id13129260...,4+,com.mlobodzinski.Stoic,"6013,6012",Health & Fitness,1312926037,stoic.,Maciej Lobodzinski,True,2021-05-29T19:53:52Z,dear stoics.\n\nthis update brings a new onboa...,6013,EUR,stoic can help you live a happier and more tra...,560803468,Maciej Lobodzinski,"Gezondheid en fitness,Lifestyle",0.00,2017-11-15T12:04:05Z,2021.23,software,328
3,https://is3-ssl.mzstatic.com/image/thumb/Purpl...,,,https://is4-ssl.mzstatic.com/image/thumb/Purpl...,https://is4-ssl.mzstatic.com/image/thumb/Purpl...,https://is4-ssl.mzstatic.com/image/thumb/Purpl...,https://apps.apple.com/nl/developer/happify-in...,,"iPhone5s-iPhone5s,iPadAir-iPadAir,iPadAirCellu...",,False,software,11.0,Happify: for Stress & Worry,"EN,FR,DE,IT,JA,PT,ZH,ES",135538688,http://www.happify.com,Gratis,4+,4.17391,23,4.17391,https://apps.apple.com/nl/app/happify-for-stre...,4+,com.happify.Happify,"6013,6012",Health & Fitness,730601963,Happify: for Stress & Worry,"Happify, Inc.",True,2021-06-11T11:18:34Z,"Everything's running faster and smoother, than...",6013,EUR,Happify's science

## Scraping App Reviews
This is where we scrape the primary data for our analysis. Note that the scraping process shown below took approximately 3 days due to the large number of reviews being requested, as well as a sleep time being built in to avoid overloading the servers.

In [ ]:
# Loop to go through entire list of apps
for app_name, app_id in zip(app_names, app_ids):
    
    # Get start time
    start = dt.datetime.now(tz=get_localzone())
    fmt= "%m/%d/%y - %T %p"
    
    # Print starting output for app
    print('---'*20)
    print('---'*20)    
    print(f'***** {app_name} started at {start.strftime(fmt)}')
    print()
    
    # Instantiate AppStore for app
    app_ = AppStore(country='us', app_name=app_name, app_id=app_id)
    
    # Scrape reviews posted since January 1, 2020 and build in random sleep time
    # between 20 and 25 seconds
    app_.review(after=dt.datetime(2020, 1, 1),
                sleep=random.randint(20,25))
    
    reviews = app_.reviews
    
    # Add keys to store information about which app each review is for
    for rvw in reviews:
        rvw['app_name'] = app_name
        rvw['app_id'] = app_id
    
    # Print update status once all reviews have been scraped for the app
    print(f"""Done scraping {app_name}. 
    Scraped a total of {app_.reviews_count} reviews.\n""")
    
    # Convert list of dicts to Pandas DataFrame and write to csv
    review_df = pd.DataFrame(reviews)
    review_df.to_csv('data/' + app_name + '.csv', index=False)
    
    # Get end time
    end = dt.datetime.now(tz=get_localzone())
    
    # Print ending output for app
    print(f"""Successfully wrote {app_name} reviews to csv
    at {end.strftime(fmt)}.\n""")
    print(f'Time elapsed for {app_name}: {end-start}')
    print('---'*20)
    print('---'*20)
    print('\n')
    
    # Wait 5 to 10 seconds to start scraping next app
    time.sleep(random.randint(5,10))

------------------------------------------------------------
------------------------------------------------------------
***** headspace-meditation-sleep started at 07/07/21 - 16:44:48 PM



2021-07-07 16:44:50,824 [INFO] Base - Initialised: AppStore('us', 'headspace-meditation-sleep', 493145008)
2021-07-07 16:44:50,825 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/headspace-meditation-sleep/id493145008
2021-07-07 16:45:15,177 [INFO] Base - [id:493145008] Fetched 14 reviews (14 fetched in total)
2021-07-07 16:46:04,332 [INFO] Base - [id:493145008] Fetched 30 reviews (30 fetched in total)
2021-07-07 16:46:53,255 [INFO] Base - [id:493145008] Fetched 47 reviews (47 fetched in total)
2021-07-07 16:47:46,306 [INFO] Base - [id:493145008] Fetched 67 reviews (67 fetched in total)
2021-07-07 16:48:35,169 [INFO] Base - [id:493145008] Fetched 83 reviews (83 fetched in total)
2021-07-07 16:49:27,500 [INFO] Base - [id:493145008] Fetched 98 reviews (98 fetched in total)
2021-07-07 16:50:18,036 [INFO] Base - [id:493145008] Fetched 110 reviews (110 fetched in total)
2021-07-07 16:51:08,098 [INFO] Base - [id:493145008] Fetched 133 reviews (133 fetched in total)
2

2021-07-07 17:54:01,460 [INFO] Base - [id:493145008] Fetched 1183 reviews (1183 fetched in total)
2021-07-07 17:54:50,306 [INFO] Base - [id:493145008] Fetched 1183 reviews (1183 fetched in total)
2021-07-07 17:55:38,949 [INFO] Base - [id:493145008] Fetched 1202 reviews (1202 fetched in total)
2021-07-07 17:56:27,746 [INFO] Base - [id:493145008] Fetched 1218 reviews (1218 fetched in total)
2021-07-07 17:57:16,491 [INFO] Base - [id:493145008] Fetched 1231 reviews (1231 fetched in total)
2021-07-07 17:58:05,423 [INFO] Base - [id:493145008] Fetched 1253 reviews (1253 fetched in total)
2021-07-07 17:58:54,324 [INFO] Base - [id:493145008] Fetched 1260 reviews (1260 fetched in total)
2021-07-07 17:59:43,269 [INFO] Base - [id:493145008] Fetched 1270 reviews (1270 fetched in total)
2021-07-07 18:00:31,944 [INFO] Base - [id:493145008] Fetched 1285 reviews (1285 fetched in total)
2021-07-07 18:01:20,659 [INFO] Base - [id:493145008] Fetched 1302 reviews (1302 fetched in total)
2021-07-07 18:02:09,

2021-07-07 19:02:33,410 [INFO] Base - [id:493145008] Fetched 2167 reviews (2167 fetched in total)
2021-07-07 19:03:22,374 [INFO] Base - [id:493145008] Fetched 2198 reviews (2198 fetched in total)
2021-07-07 19:04:11,398 [INFO] Base - [id:493145008] Fetched 2198 reviews (2198 fetched in total)
2021-07-07 19:05:00,627 [INFO] Base - [id:493145008] Fetched 2198 reviews (2198 fetched in total)
2021-07-07 19:05:49,688 [INFO] Base - [id:493145008] Fetched 2235 reviews (2235 fetched in total)
2021-07-07 19:06:38,733 [INFO] Base - [id:493145008] Fetched 2235 reviews (2235 fetched in total)
2021-07-07 19:07:27,775 [INFO] Base - [id:493145008] Fetched 2247 reviews (2247 fetched in total)
2021-07-07 19:08:16,820 [INFO] Base - [id:493145008] Fetched 2261 reviews (2261 fetched in total)
2021-07-07 19:09:05,965 [INFO] Base - [id:493145008] Fetched 2262 reviews (2262 fetched in total)
2021-07-07 19:09:54,879 [INFO] Base - [id:493145008] Fetched 2299 reviews (2299 fetched in total)
2021-07-07 19:10:43,

2021-07-07 20:11:13,618 [INFO] Base - [id:493145008] Fetched 3214 reviews (3214 fetched in total)
2021-07-07 20:12:02,702 [INFO] Base - [id:493145008] Fetched 3219 reviews (3219 fetched in total)
2021-07-07 20:12:51,598 [INFO] Base - [id:493145008] Fetched 3229 reviews (3229 fetched in total)
2021-07-07 20:13:40,328 [INFO] Base - [id:493145008] Fetched 3236 reviews (3236 fetched in total)
2021-07-07 20:14:29,811 [INFO] Base - [id:493145008] Fetched 3252 reviews (3252 fetched in total)
2021-07-07 20:15:18,781 [INFO] Base - [id:493145008] Fetched 3269 reviews (3269 fetched in total)
2021-07-07 20:16:07,543 [INFO] Base - [id:493145008] Fetched 3281 reviews (3281 fetched in total)
2021-07-07 20:16:56,456 [INFO] Base - [id:493145008] Fetched 3292 reviews (3292 fetched in total)
2021-07-07 20:17:45,455 [INFO] Base - [id:493145008] Fetched 3306 reviews (3306 fetched in total)
2021-07-07 20:18:34,587 [INFO] Base - [id:493145008] Fetched 3324 reviews (3324 fetched in total)
2021-07-07 20:19:23,

...

2021-07-08 20:47:45,970 [INFO] Base - Initialised: AppStore('us', 'slumber-fall-asleep-insomnia', 1109543953)
2021-07-08 20:47:45,970 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/slumber-fall-asleep-insomnia/id1109543953
2021-07-08 20:48:11,552 [INFO] Base - [id:1109543953] Fetched 9 reviews (9 fetched in total)
2021-07-08 20:49:02,369 [INFO] Base - [id:1109543953] Fetched 30 reviews (30 fetched in total)
2021-07-08 20:49:53,030 [INFO] Base - [id:1109543953] Fetched 56 reviews (56 fetched in total)
2021-07-08 20:50:43,929 [INFO] Base - [id:1109543953] Fetched 75 reviews (75 fetched in total)
2021-07-08 20:51:34,628 [INFO] Base - [id:1109543953] Fetched 96 reviews (96 fetched in total)
2021-07-08 20:52:25,481 [INFO] Base - [id:1109543953] Fetched 113 reviews (113 fetched in total)
2021-07-08 20:53:16,110 [INFO] Base - [id:1109543953] Fetched 136 reviews (136 fetched in total)
2021-07-08 20:54:07,125 [INFO] Base - [id:1109543953] Fetched 153 reviews (153 fetch

Done scraping slumber-fall-asleep-insomnia. 
    Scraped a total of 698 reviews.

Successfully wrote slumber-fall-asleep-insomnia reviews to csv
    at 07/08/21 - 21:23:43 PM.

Time elapsed for slumber-fall-asleep-insomnia: 0:35:58.628971
------------------------------------------------------------
------------------------------------------------------------


------------------------------------------------------------
------------------------------------------------------------
***** whats-up-a-mental-health-app started at 07/08/21 - 21:23:49 PM



2021-07-08 21:23:51,472 [INFO] Base - Initialised: AppStore('us', 'whats-up-a-mental-health-app', 968251160)
2021-07-08 21:23:51,473 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/whats-up-a-mental-health-app/id968251160
2021-07-08 21:24:15,700 [INFO] Base - [id:968251160] Fetched 8 reviews (8 fetched in total)
2021-07-08 21:25:05,213 [INFO] Base - [id:968251160] Fetched 18 reviews (18 fetched in total)
2021-07-08 21:25:53,942 [INFO] Base - [id:968251160] Fetched 22 reviews (22 fetched in total)
2021-07-08 21:25:54,353 [INFO] Base - [id:968251160] Fetched 23 reviews (23 fetched in total)


Done scraping whats-up-a-mental-health-app. 
    Scraped a total of 23 reviews.

Successfully wrote whats-up-a-mental-health-app reviews to csv
    at 07/08/21 - 21:25:54 PM.

Time elapsed for whats-up-a-mental-health-app: 0:02:04.852384
------------------------------------------------------------
------------------------------------------------------------


------------------------------------------------------------
------------------------------------------------------------
***** breethe-meditation-sleep started at 07/08/21 - 21:26:00 PM



2021-07-08 21:26:01,872 [INFO] Base - Initialised: AppStore('us', 'breethe-meditation-sleep', 920161006)
2021-07-08 21:26:01,873 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/us/app/breethe-meditation-sleep/id920161006
2021-07-08 21:26:27,150 [INFO] Base - [id:920161006] Fetched 9 reviews (9 fetched in total)
2021-07-08 21:27:18,635 [INFO] Base - [id:920161006] Fetched 27 reviews (27 fetched in total)
2021-07-08 21:28:10,731 [INFO] Base - [id:920161006] Fetched 47 reviews (47 fetched in total)
2021-07-08 21:29:01,290 [INFO] Base - [id:920161006] Fetched 66 reviews (66 fetched in total)
2021-07-08 21:29:51,780 [INFO] Base - [id:920161006] Fetched 87 reviews (87 fetched in total)
2021-07-08 21:30:42,256 [INFO] Base - [id:920161006] Fetched 101 reviews (101 fetched in total)
2021-07-08 21:31:32,940 [INFO] Base - [id:920161006] Fetched 120 reviews (120 fetched in total)
2021-07-08 21:32:23,783 [INFO] Base - [id:920161006] Fetched 140 reviews (140 fetched in total)
2021-

At this point, we have finished collecting all the data necessary for our analysis. The main analysis can be found [here]()